In [ ]:
from scipy.stats import zscore

from phd_journal.roma.plotting import *
from phd_journal.roma.combat_variations import *
from phd_journal.roma.read import *

#### I/O

In [ ]:
datasets_paths = {
    "Newcastle": "/Volumes/MMIS-Saraiv/Datasets/Newcastle/EC/features_source_ind-bands",
    "Izmir": "/Volumes/MMIS-Saraiv/Datasets/Izmir/EC/features_source_ind-bands",
    "Sapienza": "/Volumes/MMIS-Saraiv/Datasets/Sapienza/features_source_ind-bands",
}

In [ ]:
datasets_metadata_paths = {
    "Izmir": "/Volumes/MMIS-Saraiv/Datasets/Izmir/metadata.csv",
    "Sapienza": "/Volumes/MMIS-Saraiv/Datasets/Sapienza/metadata.csv",
    "Newcastle": "/Volumes/MMIS-Saraiv/Datasets/Newcastle/metadata.csv",
}

#### Read

In [ ]:
# Read datasets
datasets = {}
for dataset_name, path in datasets_paths.items():
    dataset = read_dataset(path, label=dataset_name)
    datasets[dataset_name] = dataset

In [ ]:
# Read metadata
datasets_metadata = {}
for dataset_name, path in datasets_metadata_paths.items():
    dataset = read_metadata(path, label=dataset_name)
    dataset['SITE'] = [s.split('-')[0] for s in dataset.index]
    datasets_metadata[dataset_name] = dataset

#### Before log transformation

In [ ]:
plot_mean_std_indep(datasets, datasets_metadata, log_scale=False)

In [ ]:
plot_mean_diffs(datasets)

In [ ]:
# Check normality before
check_normality(datasets, datasets_metadata)
#create_qq_plots(datasets)

#### Log transformation

In [ ]:
# Approximate normality by log transformation
datasets = {dataset_name: intra_dataset_norm(dataset, method='log') for dataset_name, dataset in datasets.items()}

In [ ]:
# Check normality after log
check_normality(datasets, datasets_metadata)
create_qq_plots(datasets)

In [ ]:
plot_mean_std_indep(datasets, datasets_metadata, log_scale=False)

In [ ]:
plot_mean_diffs(datasets, log=True)

#### Harmonisation

In [ ]:
# Selected features
columns_to_keep = ['Alpha1_Central', 'Alpha1_Occipital', 'Alpha1_Temporal',
 'Alpha2_Occipital',
 'Alpha3_Central',
 'Alpha3_Limbic',
 'Alpha3_Occipital',
 'Alpha3_Parietal',
 'Alpha3_Temporal',
 'Beta1_Occipital',
 'Beta1_Parietal',
 'Delta_Limbic',
 'Theta_Limbic',
 'Theta_Temporal']

datasets = {name: dataset[columns_to_keep] for name, dataset in datasets.items()}

In [ ]:
# Join all datasets and metadata
X = pd.concat(datasets.values())
all_metadata = pd.concat(datasets_metadata.values())
all_metadata = all_metadata.loc[X.index]  # keep only the metadata of the subjects in X
assert X.shape[0] == all_metadata.shape[0]
X = neuro_harmonize(X, all_metadata, cov_age=True, cov_gender=True, cov_education=True)
datasets_after = {dataset_name: X.loc[datasets[dataset_name].index] for dataset_name in datasets.keys()}

In [ ]:
X = intra_dataset_norm(X, method='z-score')
# Put back as it was
datasets_after_zscore = {dataset_name: X.loc[datasets[dataset_name].index] for dataset_name in datasets.keys()}

In [ ]:
# Undo log transformation
datasets_after_nolog = {dataset_name: np.power(10, dataset) for dataset_name, dataset in datasets_after.items()}

#### After harmonisation

In [ ]:
# Check normality after COMBAT
check_normality(datasets_after, datasets_metadata)
#create_qq_plots(datasets)

In [ ]:
# Plot all regions
plot_mean_std_indep(datasets_after, datasets_metadata)

In [ ]:
# Plot all regions (Z-score)
plot_mean_std_indep(datasets_after_zscore, datasets_metadata)

In [ ]:
# Plot all regions (no log)
plot_mean_std_indep(datasets_after_nolog, datasets_metadata)

In [ ]:
plot_mean_diffs(datasets_after, log=True)

In [ ]:
correlation_with_var(datasets, datasets_after, datasets_metadata, ("DIAGNOSIS", "SITE"))

In [ ]:
classification_with_var(datasets, datasets_after, datasets_metadata, ("DIAGNOSIS", "SITE"), relevant_features=None)

In [ ]:
plot_2components(datasets, datasets_after, datasets_metadata, method='pca')

In [ ]:
plot_2components(datasets, datasets_after, datasets_metadata, method='lda')

In [ ]:
plot_2components(datasets, datasets_after, datasets_metadata, method='tsne')

In [ ]:
plot_distance_matrix(datasets, datasets_after, datasets_metadata)